In [4]:
#| default_exp config

In [5]:
#| hide
%load_ext autoreload
%autoreload 2

# Configuration functions 
> This notebook introduces configuration functions designed to leverage proposed .yaml files, providing a more transparent and automated approach to artefact definition. While this method offers enhanced clarity, the option for direct definition within each notebook is also fully supported and remains practical.
>
> <span style="color: red;"> TODO: Check for possible simplifications. </span>

First, we import
- os and sys for basic access to files
- yaml for .yaml reading
- tsai.basics for using tsai artifacts

## Basic configuration and definitions

In [6]:
#| export
import os
import yaml
import sys
from tsai.basics import *
from dvats.utils import print_flush

As the yml are saved in ../nbs_pipeline/config, we will need to add '..' path

In [7]:
#| export
sys.path.append(os.path.abspath('../nbs_pipeline'))

An custom_error(message) function is added just for basic error handling

In [8]:
#| export
def custom_error(message: str):
    """
    This message raises an exception ensuring red-coloured error message is displayed
    """
    # Change to red color ANSI code
    red_start = "\033[91m"
    # Back to original color ANSI code
    reset = "\033[0m"
    raise Exception(red_start + message + reset)

## Yaml reading auxiliar functions
> Defining basics functions to read yml in order to build Attrdict's

In the version of YAML we are utilising, the '!join' constructor is not present. Therefore, it becomes necessary to define it.

In [9]:
#| export
def join_constructor(loader: yaml.Loader, node: yaml.Node) -> str:
    """
    This function adds the '!join' constructor to the YAML parsing process. 
    It constructs a sequence from the provided node and then joins the elements of the sequence into a single string.
    """
    seq = loader.construct_sequence(node)
    return ''.join(seq)

In [10]:
#| export
def recursive_attrdict(d: dict) -> AttrDict:
    """ Recursively converts a dictionary into an AttrDict, including all nested dictionaries. """
    if isinstance(d, dict):
        return AttrDict({k: recursive_attrdict(v) for k, v in d.items()})
    return d

In [11]:
#| export
def replace_includes_with_content(
    filename: str, 
    path: str = "./", 
    verbose : int = 0
) -> str:
    """
    This function processes a YAML file.
    It replaces '!include' directives with the content of the specified files. 
    It takes a filename and a path, reads the file, and iteratively replaces any '!include' directives with the content of the included files.
    The final processed content, with all '!include' directives substituted is returned as a string.
    """
    if (verbose > 0):
        print_flush("... About to replace includes with content")
    with open(path+filename, 'r', encoding='utf-8') as f:
        content = f.read()
        
        # Mientras exista una directiva !include en el contenido, sigue reemplazándola
        while "!include" in content:
            # Obtén la posición de la directiva
            start_idx = content.find('!include')
            # Encuentra el inicio y el final de las comillas que contienen el nombre del archivo
            start_quote_idx = content.find('"', start_idx) + 1
            end_quote_idx = content.find('"', start_quote_idx)
            
            # Extrae el nombre del archivo
            include_filename = content[start_quote_idx:end_quote_idx]
            
            # Lee el archivo incluido
            with open(path+include_filename, 'r', encoding='utf-8') as include_file:
                included_content = include_file.read()
            
            # Reemplaza la directiva por el contenido del archivo incluido
            content = content[:start_idx] + included_content + content[end_quote_idx+1:]
        
    return content

## Project specific yaml reading functions
> Basic configuration is read from `../config/base.yml`
> It contains the following information:
> - User Preferences: `user_preferences`
>   - General configuration: `user_preferences`
>   
>     | Key         | Description                                             | Example Value |
>     |-------------|---------------------------------------------------------|---------------|
>     | `use_wandb` | Indicates whether to use wandb for experiment tracking. | `true`        |
>   
>   - Wandb Configuration: `user_preferences.wdb`
>     
>     | Key                        | Description                                       | Example Value           |
>     |----------------------------|---------------------------------------------------|-------------------------|
>     | `user`                     | The user name for wandb.                          | `mi-santamaria`         |
>     | `project_name`             | The project name in wandb.                        | `deepvats`              |
>     | `version`                  | Specifies the version for wandb.                  | `'latest'`              |
>     | `mode`                     | Sets the mode for wandb.                          | `'offline'`             |
>     | `artifacts_path`           | The path for storing wandb artifacts.             | `'./data/wandb_artifacts'`|
>   
>   - Data configuration: `user_preferences.data`
>     
>     | Key                      | Description                                       | Example Value           |
>     |--------------------------|---------------------------------------------------|-------------------------|
>     | `folder`                 | The folder where the data is stored.              | `'~/data/'`             |
>     | `fname`                  | The file name of the dataset used.                | `'solar_4_seconds_dataset'` |
>     | `ftype`                  | The file type of the dataset.                     | `'.tsf'`                |
>     | `cols`                   | The columns to be used from the dataset.          | `[]` (all columns)      |
>     | `freq`                   | The frequency of the dataset.                     | `'4s'`                  |
>   
>   - Artifact Configuration: `user_preferences.artifact`
>     
>     | Key                      | Description                                       | Example Value           |
>     |--------------------------|---------------------------------------------------|-------------------------|
>     | `alias`                  | Alias for the resulting artifact of the run.      | `'solar_4_seconds'`     |
>     | `algorithm`              | The algorithm used in the process.                | `'mvp'`                 |
>   
>   - Directory Configuration: `user_preferences.directories`
>     
>     | Key                      | Description                                       | Example Value           |
>     |--------------------------|---------------------------------------------------|-------------------------|
>     | `tmp`                    | Folder for storing temporary files.               | `'tmp'`                 |
>     | `data`                   | Path for the data.                                | Combination of *path, *fname, *ftype |
>
> - Data Specifications: `data`
>   - Data Details: `data`
>     
>     | Key                      | Description                                       | Example Value           |
>     |--------------------------|---------------------------------------------------|-------------------------|
>     | `name`                   | The name of the data file.                        | Refers to *fname        |
>     | `path`                   | The path of the data file.                        | Refers to *data_path    |
>     | `alias`                  | The alias of the data artifact.                   | Refers to *alias        |
>     | `cols`                   | Columns of interest from the dataset.             | Refers to *cols         |
>     | `csv_config`             | Configuration for CSV files, if applicable.       | `{}` (if not required)  |
>     | `date_offset`            | Offset used for setting the date index.           | `null` (if not used)    |
>     | `date_format`            | Default date format for .tsf files.               | `'%Y-%m-%d %H:%M:%S'`   |
>     | `freq`                   | Frequency of the data.                            | Refers to *freq         |
>     | `joining_train_test`     | Linking training and testing data.                | `false`                 |
>     | `missing_values`         | Handling missing values.                          | `technique: null, constant: null` |
>     | `normalize_training`     | Indicates whether to normalize training data.     | `false`                 |
>     | `range_training`         | Specifies training ranges.                        | `null`                  |
>     | `range_testing`          | Specifies testing ranges.                         | `null`                  |
>     | `resampling_freq`        | Frequency for resampling the data.                | `null`                  |
>     | `start_date`             | Starting date for the dataset.                    | `null`                  |
>     | `test_split`             | Ratio of the test set.                            | `null`                  |
>     | `time_col`               | Column containing the timestamp.                  | `null`                  |
>   
> - Wandb Specifications: `wandb`
>     
>     | Key                      | Description                                       | Example Value           |
>     |--------------------------|---------------------------------------------------|-------------------------|
>     | `user`                   | User name in wandb specifications.                | Refers to *wdb_user     |
>     | `dir`                    | Directory for wandb.                              | Combination of '~/', *wdb_project |
>     | `enabled`                | Indicates if wandb is enabled.                    | `False`                 |
>     | `group`                  | Group for the wandb runs.                         | `null`                  |
>     | `log_learner`            | Whether to log the learner to wandb.              | `False`                 |
>     | `mode`                   | Mode for wandb operation.                         | Refers to *wdb_mode     |
>     | `project`                | The wandb project name.                           | Refers to *wdb_project  |
>     | `version`                | Version for the wandb.                            | Refers to *wdb_version  |
>     | `artifacts_path`         | Path for storing the TSArtifact in wandb.         | Refers to *artifacts_path |


In [12]:
#| export
def get_config(
        verbose: int = 0,
        filename: str = "base"
) -> AttrDict:
    """
    This function 
    - Reads the content in '../config/base.yml' and 
    - Returns its content as AttrDict.
    verbose option can be changed to True for displaying debugging messages. 
    """
    # Build file path
    filename = filename+".yaml"
    path = "./config/"
    # Debug messages
    if (verbose > 0):
        current_directory = os.getcwd()
        print_flush("Current: " + current_directory)
        print_flush("yml: "+ path + filename)
    
    # Add join constructor
    yaml.add_constructor('!join', join_constructor)
    
    if (verbose > 0): 
        print_flush("Getting content"+ path + filename)
    
    # Get file content
    full_content = replace_includes_with_content(filename, path, verbose)
    
    if (verbose > 0):
        print_flush("Load content"+ path + filename)
    
    # Load content 
    config = yaml.load(full_content, Loader=yaml.FullLoader)
    
    # Return content it as AttrDict
    return recursive_attrdict(config)

### Build the encoder artifact from already read functions
> The encoder artifact is needed for training the model.
> 
> To automatize its selection, its W&B name is built up from its latest version
> 
> <span style="color: red;"> TODO: Change for selecting specific version. Version should be selected in .yml file too </span>.

In [13]:
#| export
def build_enc_artifact(
    config  : AttrDict, 
    verbose : int  = 0,
    swv     : bool = False
) -> str:
    """
    Build enc_artifact name from config data.
    """
    version = config.user_preferences.wdb.version
    enc_artifact = config.configuration.encoder.artifacts.train.enc_prefix

    if swv:
        if not enc_artifact.endswith("-SWV"):
            enc_artifact += "-SWV"
    else:
        if enc_artifact.endswith("-SWV"):
            enc_artifact = enc_artifact[:-4]  # Remover '-SWV'
    
    if (version == 'latest'):
        enc_artifact+=":latest"
    else:
        enc_artifact=enc_artifact+":v"+version
    if (verbose > 0):
        print_flush("enc_artifact: "+enc_artifact)
    return enc_artifact


### Build project data
> Read user, project, version and dataset artifact name form already got information

In [14]:
#| export
def get_project_data(verbose : int = 0) -> [str, str, str, str]:
    """
    Retrieves project data including user, project name, version, and data name. 
    It accesses configuration settings, processes them, and optionally prints the project configuration. 
    Returns a tuple containing user, project, version, and data information.
    """
    config      = get_config()
    project     = config.wandb.project
    user        = config.wandb.user
    version     = config.wandb.version
    data_name   = config.data.alias
    if (version != "latest"):
        version = 'v' +version
    data        = data_name +":"+version
    if verbose > 0:
        dashes = '-----------'        
        print_flush(dashes+"Project configuration"+dashes)
        print_flush("user: " + user)
        print_flush("project: " + project)
        print_flush("version: " + version)
        print_flush("data: "+ data)
        print_flush(dashes+"Project configuration"+dashes)
    return user, project, version, data

### Get training artifact name

In [15]:
#| export
def get_train_artifact(user: str, project: str, data: str) -> str:
    """
    Constructs the train artifact string by combining user, project, and data information. 
    The format of the return string is 'entity/project/name:version'.
    """
    # entity/project/name:version
    train_artifact=user+'/'+project+'/'+data 
    return train_artifact

## Build dataset artifact
> Configuration file: `base`. Described above.

### Auxiliary functions

In [16]:
#| export
##############################
# 01 - DATAFRAME TO ARTIFACT #
##############################
def get_artifact_config_sd2a_get_auxiliar_variables(verbose : int) -> Tuple[str, str, str, AttrDict, bool, str]:
    """
    Retrieves auxiliary variables necessary for the dataset artifact configuration. 
    Gathers user, project, version, and data details, along with preferences for using wandb and the wandb artifacts path.
    Returns a tuple containing these elements.
    """
    user, project, version, data = get_project_data(verbose)
    config      = get_config(verbose)
    data        = config.data
    use_wandb   = config.user_preferences.use_wandb
    wandb_path  = config.wandb.artifacts_path
    return user, project, version, data, use_wandb, wandb_path

In [17]:
#| export
def get_artifact_config_sd2a_check_errors(use_wandb: str, artifact_config: AttrDict, user: str, project: str):
    """
    Checks for configuration errors in the dataset artifact settings. 
    Verifies the project and entity, and ensures appropriate settings are made for offline mode and missing values handling.
    """
    check_project_and_entity(user, project)
    if (
            use_wandb   == "offline" 
        and artifact_config.joining_train_test  == True
    ):
        custom_error("If you're using deepvats in offline mode, set joining_train_test to False")
    if (
            artifact_config.missing_values_constant is not None 
        and artifact_config.missing_values_technique is None
    ):
        custom_error("Missing values constant must be setted up only if missing_values_technique is not None. Please check base.yaml")

### Main function

In [18]:
#| export
def get_artifact_config_sd2a(verbose : int = 0) -> AttrDict:
    """
    Constructs the configuration for the dataset artifact by retrieving auxiliary variables and setting up the artifact configuration.
    Validates the configuration to ensure it meets specific criteria and returns the final artifact configuration as an AttrDict.
    """
    user, project, version, data, use_wandb, wandb_path = get_artifact_config_sd2a_get_auxiliar_variables(verbose)
    artifact_config = AttrDict(
        artifact_name           = data.alias,
        csv_config              = data.csv_config,
        data_cols               = data.cols,
        data_fpath              = data.path,
        date_format             = data.date_format,
        date_offset             = data.date_offset,
        freq                    = data.freq,
        joining_train_test      = data.joining_train_test,
        missing_values_technique= data.missing_values.technique,
        missing_values_constant = data.missing_values.constant,
        normalize_training      = data.normalize_training,
        range_training          = data.range_training,
        range_testing           = data.range_testing,
        resampling_freq         = data.resampling_freq,
        start_date              = data.start_date,
        test_split              = data.test_split,
        time_col                = data.time_col,
        use_wandb               = use_wandb,
        wandb_artifacts_path    = wandb_path
    )
    get_artifact_config_sd2a_check_errors(use_wandb, artifact_config, user, project)    
    return artifact_config

## Build MVP Encoder artifacts
> Includes also the versions for sliding window view version

The configuration files `02b-encoder_mvp` contains the following information:
### Configuration for Encoder
> - General Configuration: `configuration`
>   
>   | Key        | Description                                                 | Example Value  |
>   |------------|-------------------------------------------------------------|----------------|
>   | `job_type` | The type of job being configured.                           | `'encoder_MVP'`|
>   | `alias`    | Alias for the configuration, refers to the alias in base.  | Refers to *alias|
>
> - Wandb Configuraconfiguration.tion: `wandb`
>   
>   | Key        | Description                               | Example Value   |
>   |------------|-------------------------------------------|-----------------|
>   | `mode`     | The mode for wandb, inherited from base.  | Refers to *wdb_mode |
>   | `group`    | Specifies if the run is part of a group.  | `null`           |
>
> - Specifications Coconfiguration.nfiguration: `specifications`
>   
>   | Key                                      | Description                              | Example Value   |
>   |------------------------------------------|------------------------------------------|-----------------|
>   | `batch_size`                             | The batch size for processing.           | `1024`          |
>   | `n_epoch`                                | Number of epochs for training.           | `100`           |
>   | `mask.future`                            | Whether to mask future samples.          | `false`         |
>   | `mask.stateful`                          | Dictates if masking is stateful or not.  | `true`          |
>   | `mask.sync`                              | Mask all variables at once in series.    | `false`         |
>   | `mvp.ws1`                                | Min window size for MVP adaptable training. | `15`          |
>   | `mvp.ws2`                                | Max window size for MVP adaptable training. | `30`          |
>   | `mvp.r`                                  | Probability of masking in MVP.           | `0.710`         |
>   | `mvp.valid_size`                         | Size of the validation set proportion.   | `0.2`           |
>   | `mvp.normalize.by_sample`                | Normalization by sample indicator.       | `false`         |
>   | `mvp.normalize.use_single_batch`         | Use single batch for normalization.      | `false`         |
>   | `sliding_windows.stride`                 | Data points the window moves ahead.      | `15`            |
>   | `sliding_windows.size`                   | Window size for the sliding wiow.      | `30`            |
 g window.      | `30`            |


### Auxiliary functions for retrieving the data

In [19]:
#| export
######################
# 02b - ENCODER MVP  #
######################
def get_artifact_config_MVP_auxiliar_variables(verbose : int) -> Tuple[str, str, str, str, AttrDict, str, Tuple[float, float], AttrDict]:
    """
    Retrieves and assembles various configuration parameters and auxiliary variables for an MVP artifact. 
    Extracts user, project, version, and data details, fetches configuration settings, and constructs training artifact strings. 
    Returns a tuple containing these elements along with MVP workspace specifications and user preferences.
    """

    #Get neccesary variables
    user, project, version, data = get_project_data(verbose)
    config          = get_config(verbose, "02b-encoder_mvp")
    user_preferences = config.user_preferences
    config = config.configuration
    train_artifact_ = get_train_artifact(user,project,data)    
    mvp_ws1         = config.specifications.mvp.ws1
    mvp_ws2         = config.specifications.mvp.ws2
    mvp_ws = (mvp_ws1,mvp_ws2)
    return user, project, version, data, config, train_artifact_, mvp_ws, user_preferences

In [20]:
#| export
def get_artifact_config_MVP_auxiliar_variables_SWV(verbose : int) -> Tuple[str, str, str, str, AttrDict, str, Tuple[float, float], AttrDict]:    
    """
    Retrieves and assembles various configuration parameters and auxiliary variables for an MVP artifact. 
    Extracts user, project, version, and data details, fetches configuration settings, and constructs training artifact strings. 
    Returns a tuple containing these elements along with MVP (sliding window view) workspace specifications and user preferences.
    """
    #Get neccesary variables
    user, project, version, data = get_project_data(verbose)
    config          = get_config(verbose, "02c-encoder_mvp-sliding_window_view")
    user_preferences = config.user_preferences
    config = config.configuration
    train_artifact_ = get_train_artifact(user,project,data)    
    mvp_ws1         = config.specifications.mvp.ws1
    mvp_ws2         = config.specifications.mvp.ws2
    mvp_ws = (mvp_ws1,mvp_ws2)
    return user, project, version, data, config, train_artifact_, mvp_ws, user_preferences

### Auxiliary functions for error handling

> <span style="color: red;"> TODO: Should we allow diferent project from environment ones? Should we just avoid yml configuration for this parameters and directly get dockers environment WANDB_ENTITIY and WANDB_PROJECT environment variables? </span>.

In [21]:
#| export
def check_project_and_entity(user: str, project: str):
    """
    Checks user and project are correctly setted up comparing to environment variables
    """
    os_entity = os.environ['WANDB_ENTITY']
    os_project = os.environ['WANDB_PROJECT']
    if (os_entity != user):
        custom_error("Please check .env and base.yml: entity != user os " + os_entity + " yaml " + user)
    if (os_project != project):
        custom_error("Please check .env and base.yml: project differs os " + os_project + " yaml " + project)


In [22]:
#| export
def get_artifact_config_MVP_check_errors(
    artifact_config: AttrDict, 
    user: str, 
    project: str 
):
    """
    Avoids incompatible wandb modes configurations.
    Sets project = 'work-nbs' if wandb is not used for tracking.
    """
    check_project_and_entity(user, project)
        
    if artifact_config.use_wandb:
        if (artifact_config.analysis_mode != 'online'):
            print_flush("Changing to online analysis mode - use_wandb=true")
            artifact_config.analysis_mode = 'online'
    else:
        project = 'work-nbs'

### Main configuration functions

In [23]:
#| export
def get_artifact_config_MVP(verbose : int = 0) -> Tuple[str, str, str, str, AttrDict, str]:
    """
    Gathers and structures the MVP artifact configuration, including user, project, version, data details, and various configuration settings.
    Returns a tuple comprising user, project, version, data, the structured artifact configuration as an AttrDict, and the job type.
    """
    user, project, version, data, config, train_artifact_, mvp_ws, user_preferences = get_artifact_config_MVP_auxiliar_variables(verbose)

    artifact_config = AttrDict(
        alias                   = config.alias,
        analysis_mode           = config.wandb.mode, 
        batch_size              = config.specifications.batch_size,
        epochs                  = config.specifications.n_epoch,
        mask_future             = config.specifications.mask.future,
        mask_stateful           = config.specifications.mask.stateful,
        mask_sync               = config.specifications.mask.sync,
        mvp_ws                  = mvp_ws, 
        norm_by_sample          = config.specifications.mvp.normalize.by_sample,
        norm_use_single_batch   = config.specifications.mvp.normalize.use_single_batch,
        r                       = config.specifications.mvp.r,
        stride                  = config.specifications.sliding_windows.stride, 
        train_artifact          = train_artifact_, 
        valid_artifact          = None, 
        use_wandb               = user_preferences.use_wandb, 
        valid_size              = config.specifications.mvp.valid_size,
        w                       = config.specifications.sliding_windows.size, 
        wandb_group             = config.wandb.group
    )
    get_artifact_config_MVP_check_errors(artifact_config, user, project)
    return user, project, version, data, artifact_config, config.job_type

In [24]:
#| export
def get_artifact_config_MVP_SWV(verbose : int = 0) -> Tuple[str, str, str, str, AttrDict, str]:
    """
    Gathers and structures the MVP_SWV artifact configuration, including user, project, version, data details, and various configuration settings.
    Returns a tuple comprising user, project, version, data, the structured artifact configuration as an AttrDict, and the job type.
    """
    user, project, version, data, config, train_artifact_, mvp_ws, user_preferences = get_artifact_config_MVP_auxiliar_variables_SWV(verbose)

    artifact_config = AttrDict(
        alias                   = config.alias,
        analysis_mode           = config.wandb.mode, 
        batch_size              = config.specifications.batch_size,
        epochs                  = config.specifications.n_epoch,
        mask_future             = config.specifications.mask.future,
        mask_stateful           = config.specifications.mask.stateful,
        mask_sync               = config.specifications.mask.sync,
        mvp_ws                  = mvp_ws, 
        norm_by_sample          = config.specifications.mvp.normalize.by_sample,
        norm_use_single_batch   = config.specifications.mvp.normalize.use_single_batch,
        r                       = config.specifications.mvp.r,
        stride                  = config.specifications.sliding_windows.stride, 
        train_artifact          = train_artifact_, 
        valid_artifact          = None, 
        use_wandb               = user_preferences.use_wandb, 
        valid_size              = config.specifications.mvp.valid_size,
        w                       = config.specifications.sliding_windows.size, 
        wandb_group             = config.wandb.group
    )
    get_artifact_config_MVP_check_errors(artifact_config, user, project)
    return user, project, version, data, artifact_config, config.job_type

## Encoder DCAE configuration
> The configuration file `02a-encoder_dcae` contains the following information:

> - General Configurat: `configuration`ion
>   
>   | Key        | Description                                      | Example Value   |
>   |------------|--------------------------------------------------|-----------------|
>   | `job_type` | Type of the job for this configuration.           | `'encoder_DCAE'`|
>   | `alias`    | Alias for the run, referring to the base config.  | Refers to *alias|
>
> - Wandb Configconfiguration.uration: `wandb`
>   
>   | Key       | Description                                                 | Example Value         |
>   |-----------|-------------------------------------------------------------|-----------------------|
>   | `use`     | Indicates whether to use wandb.                             | Refers to *use_wandb  |
>   | `entity`  | Entity (user) for wandb configuration.                      | Refers to *wdb_user   |
>   | `group`   | Specifies if this run should be grouped in a wandb group.   | `null`                |
>   | `project` | Project name for wandb configuration.                       | Refers to *wdb_project|
>
> - Artifacconfiguration.ts Configuration: `artifacts`
>   
>   | Key             | Description                                                              | Example Value                                         |
>   |-----------------|--------------------------------------------------------------------------|-------------------------------------------------------|
>   | `train_prefix`  | Complete training artifact path in wandb.                                | Combination of *wdb_user, *wdb_project, and *alg      |
>   | `valid.data`    | Complete name for the validation dataset artifact (null for random).     | `null`                                                |
>   | `valid.size`    | Percentage of random items to go to validation set if `valid.data` is null. | `0.1`                                                 |
>
> - Specifications Configuration: `specifications`
>   
>   | Key            | Description                                       | Example Value   |
>   |----------------|---------------------------------------------------|-----------------|
>   | `batch_size`   | Batch size for processing.                        | `64`            |
>   | `n_epoch`      | Number of epochs to train for.                    | `200`           |
>   | `pool_szs`     | Sizes of the pooling layers in the autoencoder.   | `[2,2,4]`       |
>   | `top_k`        | Number of elements to analyse for the top losses. | `3`             |
>
> - Sliding Windows Configuration: `sliding_windows`
>   
>   | Key       | Description                             | Example Value |
>   |-----------|-----------------------------------------|---------------|
>   | `stride`  | The stride for the sliding window.      | `1`           |
>   | `size`    | Window size for the sliding window.     | `32`     configuration.     |
>
> - Autoencoder Configuration: `autoencoder`
>   
>   | Key                 | Description                                                    | Example Value  |
>   |---------------------|----------------------------------------------------------------|----------------|
>   | `delta`             | Size of the autoencoder bottleneck layer.                      | `60`           |
>   | `filters.nfs`       | Number of filters in each convolutional layer of the autoencoder. | `[64,32,16]`   |
>   | `filters.kss`       | Kernel sizes for each convolutional layer in the autoencoder.  | `[10,5,5]`     |
>   | `filters.output_size` | The output size of the autoencoder's final layer.             | `10`           |
inal layer.             | `10`           |
 | `10`           |
                  |




In [25]:
#| export
######################
# 02a - ENCODER DCAE #
######################

def get_artifact_config_DCAE(verbose : int = 0) -> Tuple[AttrDict, str]:
    """
    Constructs the configuration for the DCAE (Deep Convolutional AutoEncoder).
    It fetchs the relevant settings and assembles the artifact configuration.
    Validates the configuration to ensure correct project and entity setup and 
    returns the artifact configuration as an AttrDict along with the job type.
    """
    user, project, version, data = get_project_data(verbose)
    config = get_config(verbose, "02a-encoder_dcae")
    if verbose > 0:print_flush("Antes de leer configuration " + str(config))
    config = config.configuration
    
    artifact_config = AttrDict(
        alias               = config.alias,
        use_wandb           = config.wandb.use,
        wandb_group         = config.wandb.group,
        wandb_entity        = config.wandb.entity,
        wandb_project       = config.wandb.project,
        train_artifact      = get_train_artifact(user,project,data),
        valid_artifact      = config.artifacts.valid.data,
        valid_size          = config.artifacts.valid.size,
        w                   = config.specifications.sliding_windows.size,
        stride              = config.specifications.sliding_windows.stride,
        delta               = config.specifications.autoencoder.delta,
        nfs                 = config.specifications.autoencoder.filters.nfs,
        kss                 = config.specifications.autoencoder.filters.kss,
        output_filter_size  = config.specifications.autoencoder.filters.output_size,
        pool_szs            = config.specifications.pool_szs,
        batch_size          = config.specifications.batch_size, 
        epochs              = config.specifications.n_epoch,
        top_k               = config.specifications.pool_szs
    )
    check_project_and_entity(
        artifact_config.wandb_entity, artifact_config.wandb_project
    )
    return artifact_config, config.job_type

## Embeddings configuration
The specific configurations are in the file `03-embeddings`
It has the following information:

> - Job Type: `job_type`
>   
>   | Key       | Value         |
>   |-----------|---------------|
>   | `job_type`| `'embeddings'`|
>
> - Configuration: `configuration`
>   - Wandb Configuration: `configuration.wandb`
>   
>     | Key     | Description                                          | Example Value      |
>     |---------|------------------------------------------------------|--------------------|
>     | `group` | Whether to group this run in a wandb group.          | Refers to *emb     |
>     | `use`   | Indicates whether to use wandb.                      | Refers to *use_wandb |
>     | `entity`| Entity (user) for wandb configuration.               | Refers to *wdb_user |
>     | `project`| Project name for wandb configuration.               | Refers to *wdb_project |
>
>   - Encoder Configuration: `configuration.encoder`
>   
>     | Key                          | Description                                        | Example Value                                   |
>     |------------------------------|----------------------------------------------------|-------------------------------------------------|
>     | `artifacts.train.enc_prefix` | Prefix for the training artifacts.                 | Combination of *wdb_user, *wdb_project, and *alg |
>     | `artifacts.valid`            | If none, the validation set used to train enc is used. | `null`               |
>
>   - Specifications Configuration: `configuration.specifications`
>   
>     | Key        | Description                                  | Example Value   |
>     |------------|----------------------------------------------|-----------------|
>     | `input_ar` | Input array, if applicable.                  | `null`          |
>     | `cpu`      | Whether to use CPU instead of GPU.           | `false`         |


In [26]:
#| export
######################
# 03 - EMBEDDINGS    #
######################
def get_artifact_config_embeddings(verbose : int = 0) -> Tuple[AttrDict, str]:
    """
    Constructs the configuration for embeddings by fetching relevant settings and building the encoder artifact configuration.
    Validates the project and entity settings and returns the artifact configuration as an AttrDict, along with the job type.
    """

    config = get_config(verbose, "03a-embeddings")
    job_type=config.job_type
    version = config.user_preferences.wdb.version
    enc_artifact = build_enc_artifact(config, verbose)
    config = config.configuration
    artifact_config = AttrDict(
        use_wandb       = config.wandb.use,
        wandb_group     = config.wandb.group,
        wandb_entity    = config.wandb.entity,
        wandb_project   = config.wandb.project,
        enc_artifact    = enc_artifact,
        input_ar        = config.specifications.input_ar,
        cpu             = config.specifications.cpu
    )
    check_project_and_entity(artifact_config.wandb_entity, artifact_config.wandb_project)
    return artifact_config, job_type

In [27]:
#| export
def get_artifact_config_embeddings_SWV(verbose : int = 0) -> Tuple[AttrDict, str]:
    """
    Constructs the configuration for embeddings (sliding window view) by fetching relevant settings and building the encoder artifact configuration.
    Validates the project and entity settings and returns the artifact configuration as an AttrDict, along with the job type.
    """
    config = get_config(verbose, "03b-embeddings-sliding_window_view")
    job_type=config.job_type
    version = config.user_preferences.wdb.version
    enc_artifact = build_enc_artifact(config, verbose)
    config = config.configuration
    artifact_config = AttrDict(
        use_wandb       = config.wandb.use,
        wandb_group     = config.wandb.group,
        wandb_entity    = config.wandb.entity,
        wandb_project   = config.wandb.project,
        enc_artifact    = enc_artifact,
        input_ar        = config.specifications.input_ar,
        cpu             = config.specifications.cpu
    )
    check_project_and_entity(artifact_config.wandb_entity, artifact_config.wandb_project)
    return artifact_config, job_type


## Dimensionality Reduction Configuration
The specific configurations are in the file `04-dimensionality_reduction`
It contains the following information:

> - Job Type: `job_type`
>   
>   | Key        | Value                      |
>   |------------|----------------------------|
>   | `job_type` | `'dimensionality_reduction'`|
>
> - Configuration: `configuration`
>   - Wandb Configuration: `configuration.wandb`
>   
>     | Key     | Description                                          | Example Value      |
>     |---------|------------------------------------------------------|--------------------|
>     | `use`   | Whether to use wandb for experiment tracking.        | Refers to *use_wandb |
>     | `group` | Specifies whether to group this run in a wandb group. | `null`            |
>     | `entity`| The entity to use for wandb.                         | Refers to *wdb_user |
>     | `project`| The project to use for wandb.                       | Refers to *wdb_project |
>
>   - Encoder Configuration: `configuration.encoder`
>   
>     | Key                          | Description                                         | Example Value                                   |
>     |------------------------------|-----------------------------------------------------|-------------------------------------------------|
>     | `artifacts.train.enc_prefix` | Prefix for the training artifacts.                  | Combination of *wdb_user, *wdb_project, and *alg |
>     | `artifacts.valid`            | If none, the validation set used to train enc is used. | `null`               |
>
>   - UMAP Configuration: `configuration.encoder.umap`
>   
>     | Key             | Description                 | Example Value |
>     |-----------------|-----------------------------|---------------|
>     | `n_neighbors`   | Number of neighbors for UMAP. | `15`          |
>     | `min_dist`      | Minimum distance for UMAP.    | `0.1`         |
>     | `random_state`  | Random state for UMAP.        | `1234`        |



In [28]:
#| export
###################################
# 04 - DIMENSIONALITY REDUCTION   #
###################################
def get_artifact_config_dimensionality_reduction(verbose : int = 0) -> Tuple[AttrDict, str]:
    """
    Constructs the configuration for dimensionality reduction tasks by fetching relevant settings, including building the encoder artifact.
    Returns the artifact configuration as an AttrDict, along with the job type.
    """

    config          = get_config(verbose, "04-dimensionality_reduction")
    job_type        = config.job_type
    enc_artifact = build_enc_artifact(config, verbose)
    config = config.configuration
    try:
        if config.encoder.artifacts.dr is not None:
            config.encoder.artifacts.dr = enc_artifact
    except Exception:
        config.encoder.artifacts.dr = None
    
    artifact_config = AttrDict(
        use_wandb           = config.wandb.use, 
        wandb_group         = config.wandb.group,
        wandb_entity        = config.wandb.entity,
        wandb_project       = config.wandb.project,
        valid_artifact      = config.encoder.artifacts.valid, 
        train_artifact      = enc_artifact,
        dr_artifact         = config.encoder.artifacts.dr,
        n_neighbors         = config.encoder.umap.n_neighbors,
        min_dist            = config.encoder.umap.min_dist,
        random_state        = config.encoder.umap.random_state,
        metric              = config.encoder.umap.metric,
        cpu_flag            = config.encoder.umap.cpu_flag
    )
    return artifact_config, job_type


## XAI - shap reduction configuration
> <span style="color: red;"> TODO: Not yet implemented or configured... In progress.. </span>

The specific configurations are in the file `05-xai_shap`


In [29]:
#| export
##################
# 05 - XAI-LRP   #
##################
def get_artifact_config_xai_lrp(verbose : int = 0) -> Tuple[AttrDict, str]:
    """
    Constructs the configuration for the XAI SHAP (Explainable Artificial Intelligence using SHAP values) analysis. 
    This includes fetching relevant settings, building the encoder artifact, and assembling the artifact configuration.
    Returns the artifact configuration as an AttrDict, along with the job type.
    """
    config          = get_config(verbose, "05-xai_lrp")
    job_type        = config.job_type
    emb_artifact    = build_emb_artifact(config, verbose)
    config = config.configuration
    if verbose > 0:
        print_flush("-- config --")
        show_attrdict(config)
        print_flush("-- config --")
    artifact_config = AttrDict(
        use_wandb           = config.wandb.use, 
        wandb_group         = config.wandb.group,
        wandb_entity        = config.wandb.entity,
        wandb_project       = config.wandb.project,
        emb_artifact        = emb_artifact,
        job_type            = job_type,
        allow_val_change    = config.allow_val_change,
        stride              = config.stride
    )
    
    return artifact_config


## Use a tested configuration

[Monash benchmark](https://huggingface.co/datasets/monash_tsf) datasets used configurations

In [30]:
#| export
# Monash australian electricity demand
monash_australian_electricity_demand_0 = AttrDict(
    alias ="Monash-Australian_electricity_demand",
    fname ="australian_electricity_demand_dataset",
    #5 univariate timeseries. 0-4 can be used 1 each time
    cols =[0],
    ftype ='.tsf',
    freq ='30min', 
    time_col = None,
    mvp = AttrDict(
        batch_size = 512,
        n_epoch = 100,
        ws = [2,336], #1h-1week | TODO: Check to ensure freq sense
        stride = 48 #Day2Day TODO: Check
    ),
    dcae = AttrDict(#TODO: Check
        batch_size = 512,
        n_epoch = 100,
        stride = 48,
        w      = 224,
        delta  = 60,
        nfs    = [64, 32, 16],
        kss     = [10, 5, 5],
        output_filter_size = 10,
        top_k = [2,2,4],
        pool_szs = [2,2,4]
    )
)

In [31]:
#| export
# Monash sunspot:
monash_sunspot_0 = AttrDict(
    alias = "sunspot",
    fname = "sunspot_dataset_with_missing_values",
    ftype = ".tsf",
    cols = [],
    freq ='1d',
    time_col = None,
    mvp = AttrDict(
        batch_size = 512,
        n_epoch = 100,
        ws = [7,365], #1week-1year
        stride = 30 #1 month TODO: Check
    ),
    dcae = AttrDict(#TODO: Check
        batch_size = 512,
        n_epoch = 100,
        stride = 48,
        w      = 224,
        delta  = 60,
        nfs    = [64, 32, 16],
        kss     = [10, 5, 5],
        output_filter_size = 10,
        top_k = [2,2,4],
        pool_szs = [2,2,4]
    )
)

In [32]:
#| export
monash_solar_4_seconds_0 = AttrDict(
    alias =  'solar_4_seconds',
    fname = 'solar_4_seconds_dataset',
    ftype = '.tsf',
    freq = '4s',
    cols = [],
    time_col= None,
    mvp = AttrDict(
        batch_size = 512,
        n_epoch = 100,
        #1 4 seconds
        #15 1 min
        #450 30 min - Too small for G4-0
        #900 1h
        ws = [450,900], #1 min - 30 min (15*60=900 = 1hora intervalos 4 secs)
        stride = 1
        #stride = 450
        #stride = 10800 #1 min TODO: Check
    ),
    dcae = AttrDict(#TODO: Check
        batch_size = 512,
        n_epoch = 100,
        stride = 48,
        w      = 224,
        delta  = 60,
        nfs    = [64, 32, 16],
        kss     = [10, 5, 5],
        output_filter_size = 10,
        top_k = [2,2,4],
        pool_szs = [2,2,4]
    )
)

[Wikipedia web traffic dataset](https://zenodo.org/records/3898474)

In [33]:
#| export
wikipedia_0 = AttrDict(
    alias="Wikipedia",
    fname="kaggle_web_traffic_dataset_with_missing_values",
    cols=[0, 1, 2, 3, 4],
    ftype=".tsf",
    freq = '1d',
    time_col=None,
    mvp = AttrDict(
        batch_size = 512,
        n_epoch = 100,
        ws = [1,365], #1d-1año
        stride = 1 #TODO: Check
    ),
    dcae = AttrDict(#TODO: Check
        batch_size = 512,
        n_epoch = 100,
        stride = 48,
        w      = 224,
        delta  = 60,
        nfs    = [64, 32, 16],
        kss     = [10, 5, 5],
        output_filter_size = 10,
        top_k = [2,2,4],
        pool_szs = [2,2,4]
    )
)

[Traffic San Francisco](https://zenodo.org/records/3898445)

In [34]:
#| export
traffic_san_francisco_0 = AttrDict(
    alias="Traffic_SF",
    fname="traffic_hourly_dataset",
    ftype=".tsf",
    cols=[],
    time_col=None,
    freq='1h',
    mvp = AttrDict(
        batch_size = 512,
        n_epoch = 100,
        ws = [1,720], #1h-1week TODO: Check
        stride = 24 #1 day TODO: Check
    ),
    dcae = AttrDict(#TODO: Check
        batch_size = 512,
        n_epoch = 100,
        stride = 48,
        w      = 224,
        delta  = 60,
        nfs    = [64, 32, 16],
        kss     = [10, 5, 5],
        output_filter_size = 10,
        top_k = [2,2,4],
        pool_szs = [2,2,4]
    )
)

In [35]:
#| export
# Monash solar 10 minutes dataset
# Multi-dimensional
monash_solar_10_minutes_0 = AttrDict(
    alias ="Monash-Solar_10_minutes",
    fname ="solar_10_minutes_dataset",
    #5 univariate timeseries. 0-4 can be used 1 each time
    #137 columns (each one 1 different timeserie)
    cols =[0],
    ftype ='.tsf',
    freq ='10min',
    time_col = None,
    mvp = AttrDict(
        batch_size = 512,
        n_epoch = 100,
        #1 month 4320 -> Too big for G4 (GPU-0)
        #1 day 144 -> Ok
        #1 week 1008 -> Ok
        #15 days 2160 -> Ok
        ws = [1,2160],
        stride = 144 #1d by 1d
    ),
    dcae = AttrDict(#TODO: Check
        batch_size = 512,
        n_epoch = 100,
        stride = 48,
        w      = 224,
        delta  = 60,
        nfs    = [64, 32, 16],
        kss     = [10, 5, 5],
        output_filter_size = 10,
        top_k = [2,2,4],
        pool_szs = [2,2,4]
    )
)

### Other public datasets

#### Electricity Transformer Temperature
[ETDataset](https://paperswithcode.com/dataset/ett)

In [36]:
#| export
etth1_0 = AttrDict(
    alias="ETTh1",
    fname="ETTh1",
    ftype=".csv",
    cols=[],
    time_col=0,
    freq='1h',
    mvp = AttrDict(
        batch_size = 512,
        n_epoch = 100,
        ws = [1,720], #1 h - 1 month TODO:Check
        stride = 24 #1 day TODO: Check
    ),
    dcae = AttrDict(#TODO: Check
        batch_size = 512,
        n_epoch = 100,
        stride = 48,
        w      = 224,
        delta  = 60,
        nfs    = [64, 32, 16],
        kss     = [10, 5, 5],
        output_filter_size = 10,
        top_k = [2,2,4],
        pool_szs = [2,2,4]
    )
)

#### Stumpy
##### [Semantic segmentation TitlABP](https://stumpy.readthedocs.io/en/latest/Tutorial_Semantic_Segmentation.html)

In [37]:
#| export
stumpy_abp_0 = AttrDict(
    alias="TitlABP",
    fname="Semantic_Segmentation_TiltABP",
    ftype=".csv",
    cols=[],
    freq="1s",
    time_col=0, 
    mvp = AttrDict(
        batch_size = 512,
        n_epoch = 100,
        ws = [60,3600], #1min-1h TODO:Check
        stride = 60 #1 min TODO: Check
    ),
    dcae = AttrDict(#TODO: Check
        batch_size = 512,
        n_epoch = 100,
        stride = 48,
        w      = 224,
        delta  = 60,
        nfs    = [64, 32, 16],
        kss     = [10, 5, 5],
        output_filter_size = 10,
        top_k = [2,2,4],
        pool_szs = [2,2,4]
    )
)

##### [Multi-dimensional toy data](https://zenodo.org/records/4294932)

In [38]:
#| export
stumpy_toy_0 = AttrDict(
    alias="toy",
    fname="toy",
    ftype=".csv",
    cols=[],
    freq="1s",
    time_col=None,
    mvp = AttrDict(
        batch_size = 32,
        n_epoch = 100,
        ws = [10,30], 
        stride = 1
    ),
    dcae = AttrDict(#TODO: Check
        batch_size = 64,
        n_epoch = 200,
        stride = 1,
        w      = 30,
        delta  = 60,
        nfs    = [64, 32, 16],
        kss     = [10, 5, 5],
        output_filter_size = 10,
        top_k = [2,2,4],
        pool_szs = [2,2,4]
    )
)

### Get tested configuration function

In [39]:
#| export
tested_configs = {
    'monash_australian_electricity_demand_0': monash_australian_electricity_demand_0,
    'monash_solar_4_seconds_0': monash_solar_4_seconds_0,
    'wikipedia_0': wikipedia_0,
    'traffic_san_francisco_0': traffic_san_francisco_0,
    'monash_solar_10_minutes_0': monash_solar_10_minutes_0,
    'etth1_0': etth1_0,
    'stumpy_abp_0':  stumpy_abp_0,
    'stumpy_toy_0': stumpy_toy_0
}

In [40]:
#| export
def show_attrdict(
    dict            : AttrDict, 
    # Print options
    print_to_path   : bool          = False,
    print_path      : str           = "~/data/logs/logs.txt",
    print_mode      : str           = 'a'
):
    for key, value in dict.items():
        print_flush(f"{key}: {value}", print_to_path = print_to_path, print_path = print_path, print_mode = print_mode, print_time = print_to_path)
        print_mode = 'a'

In [41]:
#| export
def show_available_configs():
    print_flush("Available datasets: ")
    i = 0
    for key, val in tested_configs.items():
        print_flush(f"{i} - {key}")
        i+=1
    

In [42]:
#| hide 
show_available_configs()

Available datasets: 
0 - monash_australian_electricity_demand_0
1 - monash_solar_4_seconds_0
2 - wikipedia_0
3 - traffic_san_francisco_0
4 - monash_solar_10_minutes_0
5 - etth1_0
6 - stumpy_abp_0
7 - stumpy_toy_0


In [43]:
#| hide 
list(tested_configs.items())[3][0]

'traffic_san_francisco_0'

In [44]:
#| export
def show_config(id: int = 0):
    show_attrdict(list(tested_configs.items())[id][1])

In [45]:
#| hide
show_config(7)

alias: toy
fname: toy
ftype: .csv
cols: []
freq: 1s
time_col: None
mvp: {'batch_size': 32, 'n_epoch': 100, 'ws': [10, 30], 'stride': 1}
dcae: {'batch_size': 64, 'n_epoch': 200, 'stride': 1, 'w': 30, 'delta': 60, 'nfs': [64, 32, 16], 'kss': [10, 5, 5], 'output_filter_size': 10, 'top_k': [2, 2, 4], 'pool_szs': [2, 2, 4]}


In [46]:
#| export
def get_tested_config(
    id: int = 0,
    verbose = 0,
):
    if verbose > 0:show_config(id)
    return list(tested_configs.items())[id][1]
    

In [47]:
#| hide
show_attrdict(get_tested_config(7))

alias: toy
fname: toy
ftype: .csv
cols: []
freq: 1s
time_col: None
mvp: {'batch_size': 32, 'n_epoch': 100, 'ws': [10, 30], 'stride': 1}
dcae: {'batch_size': 64, 'n_epoch': 200, 'stride': 1, 'w': 30, 'delta': 60, 'nfs': [64, 32, 16], 'kss': [10, 5, 5], 'output_filter_size': 10, 'top_k': [2, 2, 4], 'pool_szs': [2, 2, 4]}


### Force tested configuration functions
#### 01 - Dataset Artifact

In [48]:
#| export
def print_colored(
    key, 
    modified_val, 
    modified, 
    both:bool=False, 
    original_val=0,
    missing_in_modified=False,
    missing_in_original=False
):
    if missing_in_modified:
        color = "\033[91m\033[1m"  # Red and bold
    elif missing_in_original:
        color = "\033[93m\033[1m"  # Orange and bold
    else:
        color = "\033[94m" if modified else ""
    reset = "\033[0m"
    
    if modified and both:
        print_flush(f"{color}{key}: {original_val}{reset} -> {modified_val}{reset}")
    elif missing_in_modified:
        print_flush(f"{color}{key} is missing in modified dict | {original_val} {reset}")
    elif missing_in_original:
        print_flush(f"{color}{key} is missing in original dict | {modified_val} {reset}")
    else:
        print_flush(f"{color}{key}: {modified_val}{reset}")

In [49]:
#| export
import pandas as pd

In [50]:
#| export 
def get_resampling_frequency(
    freq: str,
    frequency_factor:int = 1,
    verbose = 0
):
    if verbose > 0:
        print_flush("--> Frequency factor resampling frequency")
        print_flush("Freq factor: ", frequency_factor)
    freq_new = pd.to_timedelta(freq)
    freq_new = freq_new*frequency_factor
    if verbose > 0:
        print_flush("freq_original: ", freq)
        print_flush("freq_new: ", freq_new)
    suffix = "-"
    resampling_freq=""
    if freq_new.days > 0 and freq_new.seconds == pd.to_timedelta(0,'s'):
        suffix  = str(freq_new.days)+'d'
        resampling_freq = str(freq_new.days)+'D'
    elif freq_new.seconds % 3600 == 0:
        hours = (freq_new.seconds // 3600) + freq_new.days*24
        suffix = str(hours)+'h'
        resampling_freq = str(hours) + 'H'
    elif freq_new.seconds % 60 == 0: 
        minutes = (freq_new.seconds // 60) + (freq_new.days*24*60)
        suffix = str(minutes)+'m'
        resampling_freq = str(minutes)+'T'
    else: 
        seconds = freq_new.seconds + freq_new.days*24*60*60 
        suffix = str(seconds)+'s'
        resampling_freq = str(seconds)+'S'
    if verbose > 0:
        print_flush("suffix: ", suffix)
        print_flush("resampling_freq: ", resampling_freq)
        print_flush("Frequency factor resampling frequency -->")
    return (suffix, resampling_freq)

In [ ]:
get_resampling_frequency(freq = "4s",frequency_factor = 15,verbose = 2)

--> Frequency factor resampling frequency
Freq factor:  15
freq_original:  4s
freq_new:  0 days 00:01:00
suffix:  1m
resampling_freq:  1T
Frequency factor resampling frequency -->


('1m', '1T')

In [51]:
get_resampling_frequency(freq = "4s",frequency_factor = 15,verbose = 2)

--> Frequency factor resampling frequency
Freq factor:  15
freq_original:  4s
freq_new:  0 days 00:01:00
suffix:  1m
resampling_freq:  1T
Frequency factor resampling frequency -->


('1m', '1T')

In [48]:
#| export
def frequency_factor_config(
    config: AttrDict, 
    frequency_factor:int = 1,
    frequency_factor_change_alias: bool = True,
    verbose = 0
):
    if verbose > 0:
        print_flush("--> Frequency factor config")
        print_flush("Freq factor: ", frequency_factor)
        print_flush("frequency_factor_change_alias: ", frequency_factor_change_alias)
    suffix, config.resampling_freq = get_resampling_frequency(config.freq, frequency_factor, verbose)

    if frequency_factor_change_alias:
        #filename = config.data_fpath.split(".tsf", 1)[0]
        config.artifact_name = config.artifact_name+"-"+suffix
        #config.data_fpath = filename+"-"+suffix+".tsf"

    if verbose > 0:    
        print_flush("resampling_freq: ", config.freq)
        print_flush("name: ", config.artifact_name)
        print_flush("path: ", config.data_fpath)    
        print_flush("Frequency factor config -->")

In [49]:
#| export
def diff_attrdict(
    dict_original: AttrDict, 
    dict_modified: AttrDict,
    both: bool = False
):
    all_keys = set(dict_original.keys()) | set(dict_modified.keys())
    for key in all_keys:
        in_original = key in dict_original
        in_modified = key in dict_modified
        
        if in_original and in_modified:
            modified = dict_original[key] != dict_modified[key]
            print_colored(
                key, 
                modified_val = dict_modified[key], 
                modified = modified, 
                both = both, 
                original_val=dict_original[key]
            )
        elif in_original:
            # Key is missing in dict_modified
            print_colored(key, modified_val=None, modified=True, missing_in_modified=True)
        else:
            # Key is missing in dict_original
            print_colored(key, modified_val=dict_modified[key], modified=True, missing_in_original=True)


In [50]:
#| export
from copy import deepcopy
def force_artifact_config_sd2a(
    config: AttrDict,
    id:int = 0, 
    verbose = 0,
    both = False,
    frequency_factor = 1, 
    frequency_factor_change_alias = True
):
    to_set = get_tested_config(id)
    if verbose > 0:
        config_before = deepcopy(config)
        print_flush("Selecting ", list(tested_configs.items())[id][0])
    config.artifact_name = to_set.alias
    config.data_cols = to_set.cols
    config.data_fpath= "~/data/"+to_set.fname+to_set.ftype
    config.freq=to_set.freq
    config.time_col = to_set.time_col
    config.csv_config = {}
    joining_train_test= False,
    missing_values_constant= None,
    missing_values_technique= None,
    normalize_training= False,
    range_testing= None,
    range_training= None,
    resampling_freq= None,
    start_date= None,
    test_split= None,
    if frequency_factor > 1: 
        frequency_factor_config(config, frequency_factor, frequency_factor_change_alias, verbose)
    if verbose > 0:
        diff_attrdict(
            dict_original=config_before, 
            dict_modified=config, 
            both = both)

In [51]:
#| hide
sd2a_config = AttrDict(
    artifact_name= "Monash-Australian_electricity",
    csv_config= {},
    data_cols= [0],
    data_fpath= '~/data/australian_electricity_demand_dataset.csv',
    date_format= '%Y-%m-%d %H:%M:%S',
    date_offset= None,
    freq= '1s',
    joining_train_test= False,
    missing_values_constant= None,
    missing_values_technique= None,
    normalize_training= False,
    range_testing= None,
    range_training= None,
    resampling_freq= None,
    start_date= None,
    test_split= None,
    time_col= None,
    use_wandb= True,
    wandb_artifacts_path='./data/wandb_artifacts'
)

In [52]:
#| hide
force_artifact_config_sd2a(
    config = sd2a_config, 
    id = 7, 
    verbose=1, 
    both=True, 
    frequency_factor = 2, 
    frequency_factor_change_alias = True
)

Selecting  stumpy_toy_0
--> Frequency factor config
Freq factor:  2
frequency_factor_change_alias:  True
--> Frequency factor resampling frequency
Freq factor:  2
freq_original:  1s
freq_new:  0 days 00:00:02
suffix:  2s
resampling_freq:  2S
Frequency factor resampling frequency -->
resampling_freq:  1s
name:  toy-2s
path:  ~/data/toy.csv
Frequency factor config -->
start_date: None
normalize_training: False
test_split: None
date_format: %Y-%m-%d %H:%M:%S
range_testing: None
freq: 1s
artifact_name: Monash-Australian_electricity -> toy-2s
resampling_freq: None -> 2S
wandb_artifacts_path: ./data/wandb_artifacts
range_training: None
date_offset: None
csv_config: {}
data_fpath: ~/data/australian_electricity_demand_dataset.csv -> ~/data/toy.csv
joining_train_test: False
time_col: None
data_cols: [0] -> []
missing_values_technique: None
missing_values_constant: None
use_wandb: True


#### 02(bc) Encoder MVP

In [53]:
#| export
def split_artifact_string(s:string) -> tuple[string, string, string]:
    # Divide la cadena en dos partes usando ':'
    path, version = s.split(':')

    # Divide la parte del path en sus componentes
    parts = path.rsplit('/', 1)

    # Retorna los componentes separados
    return parts[0] + '/', parts[1], version

In [54]:
#| hide
result = split_artifact_string("mi-santamaria/deepvats/Monash-Australian_electricity_demand:latest")
print_flush(result)

('mi-santamaria/deepvats/', 'Monash-Australian_electricity_demand', 'latest')


In [55]:
#| export 
def force_artifact_config_mvp(
    config: AttrDict,
    id:int = 0, 
    verbose = 0,
    both = False,
    frequency_factor = 1,
    frequency_factor_change_alias = False,
):
    to_set = get_tested_config(id)
    if verbose > 0:
        config_before = deepcopy(config)
        
    force_artifact_config_sd2a(
        config = config, 
        id = id, 
        verbose = 0, 
        both = False, 
        frequency_factor = frequency_factor, 
        frequency_factor_change_alias = frequency_factor_change_alias
    )

    config.alias = to_set.alias
    config.batch_size = to_set.mvp.batch_size
    config.epochs = to_set.mvp.n_epoch
    
    config.mask_future = False 
    config.mask_stateful = True
    config.mask_sync = False
    config.norm_by_sample = False
    config.norm_use_by_single_batch = False,
    config.r = 0.71
    
    config.stride=to_set.mvp.stride
    path,_,version = split_artifact_string(config.train_artifact)
    config.train_artifact=path+config.artifact_name+":"+version

    config.valid_size = 0.2
    
    config.mvp_ws= to_set.mvp.ws
    config.w = config.mvp_ws[1]
    
    if verbose > 0:
        diff_attrdict(
            dict_original=config_before, 
            dict_modified=config, 
            both = both
        )

In [56]:
#| hide
mvp_config = AttrDict(
    alias = "Monash-Australian_electric",
    artifact_name="Monash-Australian_electricity",
    analysis_mode="online",
    batch_size=2,
    epochs=1,
    mask_future=False,
    mask_stateful=True,
    mask_sync=False,
    mvp_ws=(150, 33),
    norm_by_sample=False,
    norm_use_single_batch=False,
    r=0.71,
    stride=13,
    train_artifact="mi-santamaria/deepvats/Monash-Australian_electricity_demand:latest",
    valid_artifact=None,
    use_wandb=True,
    valid_size=0.2,
    w=30,
    wandb_group=None,
    data_cols= [],
    data_fpath= "~/data/kaggle_web_traffic_dataset_with_missing_values.tsf",
    freq= '1d',
    time_col= 'None'
)

In [57]:
#| hide 
force_artifact_config_mvp(mvp_config, 1, True, True, 5, True)

train_artifact: mi-santamaria/deepvats/Monash-Australian_electricity_demand:latest -> mi-santamaria/deepvats/solar_4_seconds-20s:latest
mask_stateful: True
norm_use_by_single_batch is missing in original dict | (False,) 
valid_artifact: None
epochs: 1 -> 100
freq: 1d -> 4s
wandb_group: None
artifact_name: Monash-Australian_electricity -> solar_4_seconds-20s
resampling_freq is missing in original dict | 20S 
norm_by_sample: False
norm_use_single_batch: False
alias: Monash-Australian_electric -> solar_4_seconds
valid_size: 0.2
csv_config is missing in original dict | {} 
batch_size: 2 -> 512
r: 0.71
data_fpath: ~/data/kaggle_web_traffic_dataset_with_missing_values.tsf -> ~/data/solar_4_seconds_dataset.tsf
analysis_mode: online
mask_sync: False
mvp_ws: (150, 33) -> [450, 900]
time_col: None -> None
stride: 13 -> 450
data_cols: []
mask_future: False
w: 30 -> 900
use_wandb: True


In [58]:
#| export 
def force_artifact_config_dcae(
    config: AttrDict,
    id:int = 0, 
    verbose = 0,
    both = False,
    frequency_factor = 1,
    frequency_factor_change_alias = False,
):
    to_set = get_tested_config(id)
    if verbose > 0:
        config_before = deepcopy(config)
        
    force_artifact_config_sd2a(
        config = config, 
        id = id, 
        verbose = 0, 
        both = False, 
        frequency_factor = frequency_factor, 
        frequency_factor_change_alias = frequency_factor_change_alias
    )

    config.alias = to_set.alias
   
    config.batch_size = to_set.dcae.batch_size
    config.epochs = to_set.dcae.n_epoch    
    config.r = 0.71
    config.stride=to_set.dcae.stride
    path,_,version = split_artifact_string(config.train_artifact)
    config.train_artifact=path+config.artifact_name+":"+version

    config.valid_size = 0.2
    config.w = to_set.dcae.w
    config.delta = to_set.dcae.delta
    config.nfs = to_set.dcae.nfs
    config.kss = to_set.dcae.kss
    config.output_filter_size = to_set.dcae.kss
    config.top_k = to_set.dcae.top_k
    config.pool_szs = to_set.dcae.pool_szs
    
    if verbose > 0:
        diff_attrdict(
            dict_original=config_before, 
            dict_modified=config, 
            both = both
        )

In [59]:
#| hide
dcae_config = AttrDict(
    alias = "Monash-Australian_electric",
    artifact_name="Monash-Australian_electricity",
    analysis_mode="online",
    batch_size=2,
    epochs=1,
    r=0,
    stride=1,
    train_artifact="mi-santamaria/deepvats/Monash-Australian_electricity_demand:latest",
    valid_artifact=None,
    use_wandb=True,
    valid_size=0.2,
    w=30,
    nfs = [],
    kss = [], 
    output_filter_size = 1,
    top_k = [],
    delta = 6,
    wandb_group=None,
    data_cols= [],
    data_fpath= "~/data/kaggle_web_traffic_dataset_with_missing_values.tsf",
    freq= '1d',
    time_col= 'None'
)

In [60]:
#| hide 
force_artifact_config_dcae(dcae_config, 1, True, True, 5, True)

train_artifact: mi-santamaria/deepvats/Monash-Australian_electricity_demand:latest -> mi-santamaria/deepvats/solar_4_seconds-20s:latest
valid_artifact: None
epochs: 1 -> 100
kss: [] -> [10, 5, 5]
freq: 1d -> 4s
wandb_group: None
artifact_name: Monash-Australian_electricity -> solar_4_seconds-20s
resampling_freq is missing in original dict | 20S 
top_k: [] -> [2, 2, 4]
pool_szs is missing in original dict | [2, 2, 4] 
alias: Monash-Australian_electric -> solar_4_seconds
valid_size: 0.2
csv_config is missing in original dict | {} 
nfs: [] -> [64, 32, 16]
batch_size: 2 -> 512
r: 0 -> 0.71
data_fpath: ~/data/kaggle_web_traffic_dataset_with_missing_values.tsf -> ~/data/solar_4_seconds_dataset.tsf
analysis_mode: online
output_filter_size: 1 -> [10, 5, 5]
time_col: None -> None
stride: 1 -> 48
data_cols: []
delta: 6 -> 60
w: 30 -> 224
use_wandb: True
